In [1]:
import  os
import  cv2
import  PIL
import  tqdm
import  shutil
import  natsort
import  warnings
import  pandas              as      pd
import  numpy               as      np
import  matplotlib.pyplot   as      plt
import  warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

from    scipy.optimize      import curve_fit
from    scipy.signal        import savgol_filter
from    scipy               import ndimage
from    PIL                 import Image
from    matplotlib.pyplot   import plot,text, cla, rcParams, text

import utils
# import  baseline_detection
# import  criteria_definition
# import  edge_superres
# import  visualization

# import tensorflow as tf
# tf.config.set_visible_devices([], 'GPU')


In [2]:
# import openpyxl, tqdm

In [3]:
# set variables
ad=r"Processed_bubble/320/S1_30per_T1_C001H001S0001"
fps=5000
cm_on_pixel_ratio= 0.0039062
error_handling_kernel_size=(5,5)
utils.make_folders(ad)

In [4]:
# model=utils.model_architecture("models/SuperRes_weights.h5")
model = utils.initiation()

angle, rotated1, red1_xs, red1_ys, red2_xs, red2_ys= utils.slope_measurement(ad)
baseline            =min(utils.find_reds(rotated1)[1])-1
name_files          =utils.load_files(ad)
img_frame           =cv2.imread(os.path.join(ad,name_files[0]))
img_frame_rotated   =utils.rotate_image(img_frame, angle)

adv_list, rec_list, contact_line_length_list, x_center_list, y_center_list, middle_angle_degree_list,processed_number_list=[],[],[],[],[],[],[]
rec_angle_point_list, adv_angle_point_list=[],[]

#utilizing morphological transformation to remove noises
kernel = np.ones(error_handling_kernel_size,np. uint8)

num_px_ratio=(0.0039062)/cm_on_pixel_ratio

In [5]:
file_number=1

In [6]:
img_drop=cv2.imread(os.path.join(ad,name_files[file_number]))
img_drop_rotated=utils.rotate_image(img_drop, angle)

#drop diff
diff_img=cv2.absdiff(img_drop_rotated, img_frame_rotated)

#drop cropping
BaseL=utils.Baseline(baseline, middle_drop_height=25,drop_start_height=3*3, object_detection_threshold=40)
drop_reflection,*dim=BaseL.drop_cropping(diff_img, x_left_margin=30, x_right_margin=60,y_up_margin=10)
just_drop = diff_img[dim[2]:baseline,dim[0]:dim[1],:]

#super resolution    
upscaled_image=utils.upscale_image(model, cv2.cvtColor(just_drop.astype('uint8'), cv2.COLOR_BGR2RGB))


#utilizing morphological transformation to remove noises
upscaled_image=cv2.morphologyEx(np.array(upscaled_image), cv2.MORPH_CLOSE, kernel)

#keeping just external pixels as droplet curvature
i_list, j_list =utils.edge_extraction( upscaled_image, thr=40)

#extracting the desired number of pixels as input of the polynomial fitting 
left_number_of_pixels=int(150*num_px_ratio)
right_number_of_pixels=int(65*num_px_ratio)
i_left, j_left=utils.advancing_pixel_selection(i_list,j_list, left_number_of_pixels=left_number_of_pixels)
i_right, j_right=utils.receding_pixel_selection(i_list,j_list, right_number_of_pixels=right_number_of_pixels)

#rotation for fitting, it can increase the accuracy to rotate 90 degrees then fit the polynomial
i_left_rotated,j_left_rotated=j_left,i_left       
i_right_rotated,j_right_rotated=j_right,i_right   

left_polynomial_degree=3
right_polynomial_degree=2

i_poly_left_rotated, j_poly_left_rotated    = utils.poly_fitting(i_left_rotated,j_left_rotated,polynomial_degree=left_polynomial_degree,line_space=left_number_of_pixels)
i_poly_right_rotated, j_poly_right_rotated  = utils.poly_fitting(i_right_rotated,j_right_rotated,polynomial_degree=right_polynomial_degree,line_space=right_number_of_pixels)

right_angle_degree,right_angle_point        = utils.right_angle(i_poly_right_rotated, j_poly_right_rotated,1)
left_angle_degree,left_angle_point          = utils.left_angle(i_poly_left_rotated, j_poly_left_rotated,1)


In [7]:
from ultralytics import YOLO

yolo_m = YOLO("models/best.pt")
_img_drop = img_drop[-200:,:,:]
results = yolo_m(_img_drop, verbose=False)
x1,y1,x2,y2 = np.array(results[0].boxes.xyxy[:, :].cpu().numpy(), dtype=np.int32)[0]
x1,y1,x2,y2 = x1,y1,x2,y2
just_drop2 = _img_drop[y1-10:y2-7,x1-10:x2+10,:]
just_drop2 = cv2.bitwise_not(just_drop2)


import matplotlib
matplotlib.use('TkAgg')
plt.imshow(just_drop)
plt.show()
plt.imshow(just_drop2)
plt.show()

In [8]:
i_list, j_list, i_left, j_left, i_right, j_right, j_poly_left, i_poly_left, j_poly_right, i_poly_right, x_cropped, i_poly_left_rotated, j_poly_left_rotated, i_poly_right_rotated, j_poly_right_rotated = utils.base_function_process(ad,name_files,file_number,
                                                                                                                                                                                                                                              angle = angle, img_frame_rotated = img_frame_rotated, baseline = baseline, model = model, kernel = kernel, num_px_ratio=num_px_ratio,yolo_m=yolo_m)
distance = (x_cropped) * 3
address=os.path.join(ad,'SR_edge',str(name_files[file_number]))
adv, rec,rec_angle_point, adv_angle_point, contact_line_length, x_center, y_center, middle_angle_degree=utils.visualize(address, 
                                                                                                                            distance+np.array(i_list),j_list,distance+np.array(i_left),j_left,distance+np.array(i_right),j_right,
                                                                                                                            j_poly_left,distance+np.array(i_poly_left),j_poly_right,distance+np.array(i_poly_right),x_cropped,
                                                                                                                            distance+np.array(i_poly_left_rotated), j_poly_left_rotated, distance+np.array(i_poly_right_rotated),
                                                                                                                            j_poly_right_rotated, cm_on_pixel=cm_on_pixel_ratio, middle_line_switch=1)

processed_number_list.append(int(name_files[file_number].split(".")[0].split("S0001")[-1]))
adv_list.append(adv)
rec_list.append(rec)
adv_angle_point_list.append(adv_angle_point)
rec_angle_point_list.append(rec_angle_point)
contact_line_length_list.append(contact_line_length)
x_center_list.append(x_center)
y_center_list.append(y_center)
middle_angle_degree_list.append(middle_angle_degree)


In [9]:
i_list, j_list, i_left, j_left, i_right, j_right, j_poly_left, i_poly_left, j_poly_right, i_poly_right, x_cropped, i_poly_left_rotated, j_poly_left_rotated, i_poly_right_rotated, j_poly_right_rotated = utils.base_function_process(ad,name_files,file_number,
                                                                                                                                                                                                                                              angle = angle, img_frame_rotated = img_frame_rotated, baseline = baseline, model = model, kernel = kernel, num_px_ratio=num_px_ratio,yolo_m=yolo_m)
distance = (x_cropped) * 3
address=os.path.join(ad,'SR_edge',str(name_files[file_number]))
adv, rec,rec_angle_point, adv_angle_point, contact_line_length, x_center, y_center, middle_angle_degree=utils.visualize(address, 
                                                                                                                            distance+np.array(i_list),j_list,distance+np.array(i_left),j_left,distance+np.array(i_right),j_right,
                                                                                                                            j_poly_left,distance+np.array(i_poly_left),j_poly_right,distance+np.array(i_poly_right),x_cropped,
                                                                                                                            distance+np.array(i_poly_left_rotated), j_poly_left_rotated, distance+np.array(i_poly_right_rotated),
                                                                                                                            j_poly_right_rotated, cm_on_pixel=cm_on_pixel_ratio, middle_line_switch=1)

processed_number_list.append(int(name_files[file_number].split(".")[0].split("S0001")[-1]))
adv_list.append(adv)
rec_list.append(rec)
adv_angle_point_list.append(adv_angle_point)
rec_angle_point_list.append(rec_angle_point)
contact_line_length_list.append(contact_line_length)
x_center_list.append(x_center)
y_center_list.append(y_center)
middle_angle_degree_list.append(middle_angle_degree)


In [10]:
x_center_list

[np.float64(0.8310050810721015), np.float64(0.8310050810721015)]

In [ ]:
if os.path.isfile(os.path.join(ad,"error_log.txt")):
    os.remove(os.path.join(ad,"error_log.txt"))

#def folder_pro
for file_number in tqdm.tqdm(range(1, len(name_files))):
    try:
        i_list, j_list, i_left, j_left, i_right, j_right, j_poly_left, i_poly_left, j_poly_right, i_poly_right, x_cropped, i_poly_left_rotated, j_poly_left_rotated, i_poly_right_rotated, j_poly_right_rotated = utils.base_function_process(ad,name_files,file_number,
                                                                                                                                                                                                                                              angle = angle, img_frame_rotated = img_frame_rotated, baseline = baseline, model = model, kernel = kernel, num_px_ratio=num_px_ratio,yolo_m=yolo_m)
        distance = (x_cropped) * 3
        address=os.path.join(ad,'SR_edge',str(name_files[file_number]))
        adv, rec,rec_angle_point, adv_angle_point, contact_line_length, x_center, y_center, middle_angle_degree=utils.visualize(address, 
                                                                                                                                    distance+np.array(i_list),j_list,distance+np.array(i_left),j_left,distance+np.array(i_right),j_right,
                                                                                                                                    j_poly_left,distance+np.array(i_poly_left),j_poly_right,distance+np.array(i_poly_right),x_cropped,
                                                                                                                                    distance+np.array(i_poly_left_rotated), j_poly_left_rotated, distance+np.array(i_poly_right_rotated),
                                                                                                                                    j_poly_right_rotated, cm_on_pixel=cm_on_pixel_ratio, middle_line_switch=1)
        
        processed_number_list.append(int(name_files[file_number].split(".")[0].split("S0001")[-1]))
        adv_list.append(adv)
        rec_list.append(rec)
        adv_angle_point_list.append(adv_angle_point)
        rec_angle_point_list.append(rec_angle_point)
        contact_line_length_list.append(contact_line_length)
        x_center_list.append(x_center)
        y_center_list.append(y_center)
        middle_angle_degree_list.append(middle_angle_degree)

    except Exception as e:
        # Append the error message to a log file
        with open(os.path.join(ad,"error_log.txt"), "a") as log_file:
            log_file.write(f"File name {os.path.join(ad,name_files[file_number])} with shape of :{cv2.imread(os.path.join(ad,name_files[file_number])).shape}" + "\n")

vel=[]
for i in range(len(x_center_list)-1):
    vel=vel+[x_center_list[i+1]-x_center_list[i]]

vel=np.array(vel)*fps

df=pd.DataFrame([processed_number_list, np.arange(0, 1/fps*len(vel), 1/fps), x_center_list, adv_list,rec_list,contact_line_length_list, y_center_list, middle_angle_degree_list, vel]).T
df=df[:-1]

df.columns=['file number', "time (s)", 'x_center (cm)', 'adv (degree)', 'rec (degree)', 'contact_line_length (cm)', 'y_center (cm)', 'middle_angle_degree (degree)', 'velocity (cm/s)']
filter_size=9

df["adv (degree)"]=savgol_filter(df["adv (degree)"], filter_size, 2)
df["rec (degree)"]=savgol_filter(df["rec (degree)"], filter_size, 2)
df["contact_line_length (cm)"]=savgol_filter(df["contact_line_length (cm)"], filter_size, 2)
df["y_center (cm)"]=savgol_filter(df["y_center (cm)"], filter_size, 2)
df["middle_angle_degree (degree)"]=savgol_filter(df["middle_angle_degree (degree)"], filter_size, 2)
df["velocity (cm/s)"]=savgol_filter(df["velocity (cm/s)"], filter_size, 2)
df.to_excel(os.path.join(ad,'SR_result','result.xlsx'))


 37%|███▋      | 358/962 [00:31<00:51, 11.70it/s]

In [ ]:
import matplotlib
matplotlib.use('TkAgg')

plt.figure(figsize=(8, 5))  # Adjust the size of the figure
plt.plot(df["x_center (cm)"], df["adv (degree)"], label="Advancing Angle", color="blue", linestyle="-", linewidth=2)
plt.plot(df["x_center (cm)"], df["rec (degree)"], label="Receding Angle", color="green", linestyle="-", linewidth=2)

# Adding titles and labels
plt.title("Advancing and Receding Angles Over Time", fontsize=14)
plt.xlabel("x_center (cm)", fontsize=12)
plt.ylabel("Angle (degrees)", fontsize=12)

# Customizing the grid
plt.grid(color="gray", linestyle="--", linewidth=0.5, alpha=0.7)

plt.ylim(0, 180)

# Adding a legend
plt.legend(fontsize=10, loc="best")

# Improving axis ticks
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Display the plot
plt.tight_layout()  # Ensures everything fits well
plt.show()
